# Working with Mosaics

## Intro

The NASA-IMPACT TiTiler fork adds the ability to generate and store a MosaicJSON entity in the service, and generate tiles from this stored MosaicJSON. For very large MosaicJSON files, this has much better performance that the existing tiles endpoint that takes a URL to a MosaicJSON file as a parameter.

There are three ways to create a MosaicJSON:

- provide a valid MosaicJSON body
- provide a list of URIs to construct a MosaicJSON from
- provide a STAC API endpoint and query, from which the results will be used to contruct a MosaicJSON

Construction from a list of URIs or a STAC API endpoint is limited to only allow 100 files to be mosaiced, since this is a relatively expensive and time-consuming operation to run on the server.

All three of these creation sources use the same endpoint, `/mosaicjson/mosaics`, and distinguish the source with the `Content-Type` header in the request.

In [1]:
# Uncomment this line if you need to install the dependencies
#!pip install requests folium

In [2]:
# imports and helper functions

import requests
import json
import folium


# the root URL for the service
root_url = "https://h9su0upami.execute-api.us-east-1.amazonaws.com"

## Create with MosaicJSON

This example creates a mosaicjson resource from an existing MosaicJSON file.

Content-Type header is set to `application/vnd.mosaicjson+json`

This is also the default behavior if the request Content-Type header is not set or is set to `application/json` or `application/json; charset=utf-8`.

Below is a MosaicJSON description as a dict. When constructing a MosaicJSON definition, it is necessary to use a library such as [cogeo-mosaic](https://github.com/developmentseed/cogeo-mosaic).

In [3]:
mosaicjson_dict = {
    "mosaicjson": "0.0.2",
    "name": "my_fabulous_mosaic",
    "description": "A fabulous mosaic of the Sahara Desert in northern Chad",
    "version": "1.0.0",
    "attribution": "Contains modified Copernicus Sentinel data 2021",  
    "minzoom": 8,
    "maxzoom": 14,
    "quadkey_zoom": 8,
    "bounds": [
      19.066830301965645,
      19.79785386241398,
      21.2739508724601,
      21.703408250526167
    ],
    "center": [
      20.17039058721287,
      20.750631056470073,
      8
    ],
    "tiles": {
      "12221101": [
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CH/2021/4/S2B_34QCH_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CJ/2021/4/S2B_34QCJ_20210420_0_L2A/TCI.tif"
      ],
      "12221110": [
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CH/2021/4/S2B_34QCH_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/DH/2021/4/S2B_34QDH_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CJ/2021/4/S2B_34QCJ_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/DJ/2021/4/S2B_34QDJ_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/EJ/2021/4/S2B_34QEJ_20210420_0_L2A/TCI.tif"
      ],
      "12221111": [
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/DJ/2021/4/S2B_34QDJ_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/EJ/2021/4/S2B_34QEJ_20210420_0_L2A/TCI.tif"
      ],
      "12221103": [
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CH/2021/4/S2B_34QCH_20210420_0_L2A/TCI.tif"
      ],
      "12221112": [
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CH/2021/4/S2B_34QCH_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/DH/2021/4/S2B_34QDH_20210420_0_L2A/TCI.tif"
      ]
    }
}

POST it, using the header `Content-Type: application/vnd.mosaicjson+json` that describes to the endpoint that the body is a MosaicJSON file.

In [4]:
r = requests.post(
            url=f"{root_url}/mosaicjson/mosaics",
            headers={
                "Content-Type": "application/vnd.titiler.mosaicjson+json",
            },
            json=mosaicjson_dict)

The response body is the same for all responses, and will be described later, particularly the available link relations.

In [5]:
print("Response:")
print(json.dumps(r.json(), indent=2))

Response:
{
  "id": "06aca4cd-a8b4-4ade-b163-ccec05ce0fcd",
  "links": [
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/06aca4cd-a8b4-4ade-b163-ccec05ce0fcd",
      "rel": "self",
      "type": "application/json",
      "title": "Self"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/06aca4cd-a8b4-4ade-b163-ccec05ce0fcd/mosaicjson",
      "rel": "mosaicjson",
      "type": "application/json",
      "title": "MosiacJSON"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/06aca4cd-a8b4-4ade-b163-ccec05ce0fcd/tilejson.json",
      "rel": "tilejson",
      "type": "application/json",
      "title": "TileJSON"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/06aca4cd-a8b4-4ade-b163-ccec05ce0fcd/tiles/{z}/{x}/{y}",
      "rel": "tiles",
      "type": "application/json",
      "title": "Tile

## Create with list of URLs

This creates a MosaicJSON from a list of URLs. 

The `Content-Type` to set in this request is `application/vnd.titiler.urls+json`

Parameters:

- **urls** - the root endpoint (Landing Page) for the STAC API. Required, no default. This list must not contain more than 100 URLs.
- **name** - the name field for the resulting MosaicJSON entity. Defaults to `null`.
- **description** - the description field for the resulting MosaicJSON entity. Defaults to `null`.
- **attribution** - the attribution field for the resulting MosaicJSON entity. Defaults to `null`.
- **minzoom** - the forced minimum zoom for the mosaic.
- **maxzoom** - the forced maximum zoom for the mosaic.

In [6]:
r = requests.post(
        url=f"{root_url}/mosaicjson/mosaics",
        headers={
            "Content-Type": "application/vnd.titiler.urls+json",
        },
        json={ 
                "name": "my_fabulous_mosaic",
                "description": "A fabulous mosaic of the Sahara Desert in northern Chad",
                "attribution": "Contains modified Copernicus Sentinel data 2021",  
                "urls": [
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CH/2021/4/S2B_34QCH_20210420_0_L2A/TCI.tif"
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CH/2021/4/S2B_34QCH_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CJ/2021/4/S2B_34QCJ_20210420_0_L2A/TCI.tif"
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/CJ/2021/4/S2B_34QCJ_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/DH/2021/4/S2B_34QDH_20210420_0_L2A/TCI.tif"
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/DH/2021/4/S2B_34QDH_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/DJ/2021/4/S2B_34QDJ_20210420_0_L2A/TCI.tif",
        "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/34/Q/EJ/2021/4/S2B_34QEJ_20210420_0_L2A/TCI.tif"
               ],
                "minzoom": "3",
                "maxzoom": "20"
        }
    )

In [7]:
print("Response:")
print(json.dumps(r.json(), indent=2))

Response:
{
  "id": "48bed98b-57e8-4006-9d40-c1bdb26b977e",
  "links": [
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/48bed98b-57e8-4006-9d40-c1bdb26b977e",
      "rel": "self",
      "type": "application/json",
      "title": "Self"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/48bed98b-57e8-4006-9d40-c1bdb26b977e/mosaicjson",
      "rel": "mosaicjson",
      "type": "application/json",
      "title": "MosiacJSON"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/48bed98b-57e8-4006-9d40-c1bdb26b977e/tilejson.json",
      "rel": "tilejson",
      "type": "application/json",
      "title": "TileJSON"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/48bed98b-57e8-4006-9d40-c1bdb26b977e/tiles/{z}/{x}/{y}",
      "rel": "tiles",
      "type": "application/json",
      "title": "Tile

## Create with STAC API query

This is the same body that would be POST'ed to the STAC API Item Search endpoint (`/search`), with the addition of these fields:

- **stac_api_root** - the root endpoint (Landing Page) for the STAC API. Required, no default.
- **name** - the name field for the resulting MosaicJSON entity. Defaults to `null`.
- **description** - the description field for the resulting MosaicJSON entity. Defaults to `null`.
- **attribution** - the attribution field for the resulting MosaicJSON entity. Defaults to `null`.
- **asset_name** - the name of the asset in each STAC Item to use for the mosaic. Defaults to `visual`.

This query must not return more than 100 items.

The `Content-Type` header to set for this request is `application/vnd.titiler.stac-api-query+json`

In [8]:
r = requests.post(
        url=f"{root_url}/mosaicjson/mosaics",
        headers={
            "Content-Type": "application/vnd.titiler.stac-api-query+json",
        },
        json={
          "stac_api_root": "https://earth-search.aws.element84.com/v0",
          "name": "foo",
          "description": "bar",
          "attribution": "shmattribution",
          "asset_name": "visual",
          "collections": [
            "sentinel-s2-l2a-cogs"
          ],
          "datetime": "2021-04-20T00:00:00Z/2021-04-21T02:00:00Z",
          "bbox": [
            20,
            20,
            21,
            21
          ]
        }
    )

In [9]:
response_body = r.json()

print("Response:")
print(json.dumps(response_body, indent=2))

Response:
{
  "id": "23ed9cd2-fc82-4261-bad2-d16417522ee6",
  "links": [
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6",
      "rel": "self",
      "type": "application/json",
      "title": "Self"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6/mosaicjson",
      "rel": "mosaicjson",
      "type": "application/json",
      "title": "MosiacJSON"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6/tilejson.json",
      "rel": "tilejson",
      "type": "application/json",
      "title": "TileJSON"
    },
    {
      "href": "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6/tiles/{z}/{x}/{y}",
      "rel": "tiles",
      "type": "application/json",
      "title": "Tile

The `Location` header in the response contains the URI of the newly-created Mosaic resource:

In [10]:
location_header_url = r.headers["location"]
print(location_header_url)

https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6


In [11]:
def get_link_by_rel(response_body: dict, rel: str) -> str:
    return next((x["href"] for x in response_body["links"] if x["rel"] == rel), None)

In [12]:
self_link = get_link_by_rel(response_body, "self")
print(f"self href: {self_link}")

self href: https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6


The tilejson link can be used to retrieve a TileJSON file describing the tiles endpoint. This URL accepts the query parameters `tile_scale` (e.g, `2` for Retina displays) and `tile_format` (e.g., png, jpg) and adds these parameters to the tiles URL. 

In [13]:
tilejson_link = get_link_by_rel(response_body, "tilejson")
print(f"tilejson href: {tilejson_link}")

tilejson href: https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6/tilejson.json


In [14]:
requests.get(url=tilejson_link, params = { "tile_format":"jpg", "tile_scale": 2 }).json()["tiles"][0]

'https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6/tiles/{z}/{x}/{y}@2x.jpg?'

The `mosaicjson` link can be used to retrieve the entire mosaicjson, but this is rarely necessary and expensive to perform. In most cases, the `tiles` link will be used isntead.

In [15]:
mosaicjson_link = get_link_by_rel(response_body, "mosaicjson")
print(f"mosaicjson href: {mosaicjson_link}")

mosaicjson href: https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6/mosaicjson


In [16]:
tiles_link = get_link_by_rel(response_body, "tiles")
print(f"tiles href: {tiles_link}")

tiles href: https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/23ed9cd2-fc82-4261-bad2-d16417522ee6/tiles/{z}/{x}/{y}


Now we'll use the `tiles` link, which uses the simple `xyz` tile URL scheme and generate a map. More complex tile URL formats can be constructed by passing parameters to the mosaic tilejson endpoint (shown above). The tilejson endpoint accepts the same parameters as the non-mosaicjson tilejson endpoint. 

In [17]:
m = folium.Map(
    zoom_start=10,
    location=[20, 20]
)

tile_layer = folium.TileLayer(
    tiles=tiles_link,
    min_zoom=3,
    max_zoom=20,
    opacity=1,
    attr="Sentinel-2 L2A"
)
tile_layer.add_to(m)

m